In [1]:
#cluster blog posts based on word frequencies
#might be able to determine if there are groups (or clusters)
#of blogs that write about similar subjects or write in similar styles

import feedparser
import re

#returns title and diction of word counts for an RSS feed
def get_word_counts(url):
    #parse the feed
    d = feedparser.parse(url)
    wc = {}
    
    #loop over all the entries
    for e in d.entries:
        if 'summary' in e:
            summary = e.summary
        else:
            summary = e.description
            
        #extract a list of words
        words = get_words(e.title+' '+summary)
        for word in words:
            wc.setdefault(word,0)
            wc[word]+=1
    return d.feed.title, wc

In [2]:
#rss feeds always have a title and list of entries
#each entry usually has a summary or description tag
#which contains the actual text of the entries
#get_word_counts passes this summary to get_words
#which strips out all of the HTML and splits words by non-alphabetical
#characters, returning them as a list

In [3]:
def get_words(html):
    #remove all HTML tags
    txt = re.compile(r'<[^>]+>').sub('',html)
    
    #split words by all non-alpha characters
    words = re.compile(r'[^A-Z^a-z]+').split(txt)
    
    #convert to lowercase
    return [word.lower() for word in words if word!='']

In [4]:
#need a list of feeds to work from, data/feedlist.txt
#is a plain text file with a URL on each line

#first part of the code loops over every line in feedlist.txt
#generates word counts for each blog as well as number of blogs
#that each word appears (apcount)

def generate_feed_vector():
    apcount = {}
    wordcounts = {}
    for feedurl in open('data/feedlist.txt'):
        title, wc = get_word_counts(feedurl)
        wordcounts[title] = wc
        for word,count in wc.items():
            apcount.setdefault(word,0)
            if count>1:
                apcount[word]+=1
                
    #next, generate list of words that will be used in other counts
    #for each blog
    #don't want words that are too common, i.e. 'the', or too rare
    #so set upper and lower bounds
    #here we set 10% as lower bound and 50% as upper bound
    wordlist = []
    for w, bc in apcount.items():
        frac = bc/len(feedlist)
        if frac>0.1 and frac<0.5:
            wordlist.append(w)
            
    #final step is to use the list of words and list of blogs
    #to create a text file containing a big matrix of all word counts
    #for each of the blogs
    out = file('data/blogdata.txt', 'w')
    out.write('Blog')
    for word in wordlist:
        out.write('\t%s' % word)
    out.write('\n')
    for blog, wc in wordcounts.items():
        out.write(blog)
        for word in wordlist:
            if word in wc:
                out.write('\t%d' % wc[word])
            else:
                out.write('\t0')
        out.write('\n')

In [5]:
#generate_feed_vector()

In [6]:
#hierarcical clustering builds up a hierarchy of groups by
#continuously merging the two most similar groups
#each of these groups starts as a single item, i.e. individual blog

#this method calculates the distances between every pair of groups
#closest ones merged together to form a new group

#similarity of items represented by their relative locations, the 
#closer two items are, the more similar they are
#when two items, A and B, merge into a single group, their new location
#is halfway between the two items

#can also view as a tree, with each blog being a leaf and each
#'merge' of the group being the two leaves joining
#think of graph draw from leaf to root 

#this function reads the top row into list of column names
#the leftmost column into a list of row_names 
#then puts all data into big list where every item in the list
#is the data for that row
#the count for a cell can be referenced by its row and column in data
#which corresponds to indices of row_names and col_names list

def read_file(filename):
    lines = [line for line in open(filename)]
    
    #first line is column titles
    col_names = lines[0].strip().split('\t')[1:]
    row_names = []
    data = []
    for line in lines[1:]:
        p = line.strip().split('\t')
        #first column in each row is the row_name
        row_names.append(p[0])
        #data for this row is the remainder of the row
        data.append([float(x) for x in p[1:]])
    return row_names, col_names, data

In [7]:
#need to define closeness using Euclidean distance and/or 
#Pearson correlation

#some blogs contain more entries or longer entries than others
#therefore contain more words overall
#Pearson correlation will correct for this as it tries to determine
#how well two sets of data fit onto a straight line

#remember, pearson correlation = 1.0 when two items match perfectly
#and 0 when there is no relationship at all

#we want a lower number when things are more correlated, so we do
#1-num/den

import math
def pearson(v1, v2):
    #sums
    sum1=sum(v1)
    sum2=sum(v2)
    
    #sum of the squares
    sos1=sum([pow(v,2) for v in v1])
    sos2=sum([pow(v,2) for v in v2])
    
    #sum of the products
    sop=sum([v1[i]*v2[i] for i in range(len(v1))])
    
    #calculate 'r', the Pearson score
    num=sop-(sum1*sum2/len(v1))
    den=math.sqrt((sos1-pow(sum1,2)/len(v1))*(sos2-pow(sum2,2)/len(v1)))
    if den==0:
        return 0
    
    return 1-(num/den)

In [8]:
#each cluster is either a point in the tree with two branches or an 
#endpoint associated with an actual row from the dataset

#each cluster also contains data about its location which is either the
#row data for the endpoints or the merged data from its two branches
#for the other node types

#the bicluster class has all of these properties used to represent
#the hierarchical tree

class Bicluster:
    def __init__(self, vec, left=None, right=None, distance=0.0, id=None):
        
        self.vec=vec
        self.left=left
        self.right=right
        self.distance=distance
        self.id=id

In [9]:
#hierarchical clustering begins by creating group of clusters that are
#just the original items

#main loop of the function searches for two best matches by trying every possible
#pair and calculating their correlation

#the best pair of clusters is merged into a single cluster
#the data for this new cluster is the average of the data for the two old 
#clusters

#this is repeated until only one cluster remains

#this takes time, so it's a good idea to store correlation results for each pair
#since they will be calculated repeatedly

def h_cluster(rows, distance=pearson):
    distances = {}
    curr_clust_id = -1
    
    #clusters are initially just the rows
    clust = [Bicluster(rows[i],id=1) for i in range(len(rows))]
    
    while len(clust)>1:
        lowest_pair=(0,1)
        closest=distance(clust[0].vec, clust[1].vec)
        
        #loop through each pair looking for the smallest distance
        for i in range(len(clust)):
            for j in range(i+1, len(clust)):
                #distances is the cache of distance calculations
                if (clust[i].id,clust[j].id) not in distances:
                    distances[(clust[i].id,clust[j].id)]=distance(clust[i].vec,clust[j].vec)
                    
                d=distances[(clust[i].id,clust[j].id)]
                
                if d<closest:
                    closest=d
                    lowest_pair=(i,j)
                    
        #calculate the average of two clusters
        merge_vec = [(clust[lowest_pair[0]].vec[i]+clust[lowest_pair[1]].vec[i])/2 \
                     for i in range(len(clust[0].vec))]
        
        #create new cluster
        new_cluster = Bicluster(merge_vec, left=clust[lowest_pair[0]],
                                           right=clust[lowest_pair[1]],
                                           distance=closest,
                                           id=curr_clust_id)
        
        #cluster ids that weren't in the original set are negative
        curr_clust_id -= 1
        del clust[lowest_pair[1]]
        del clust[lowest_pair[0]]
        clust.append(new_cluster)
        
    return clust[0]

In [10]:
#because each cluster references the two clusters that were merged
#to create it, the final cluster returned by this function can be
#searched recursively to recreate all the clusters and their end nodes

#to run it, call h_cluster on the data

blog_names, words, data = read_file('data/blogdata_dl.txt')
clust = h_cluster(data)

In [19]:
#to print them out to view

"""
THIS DOESN'T WORK FOR SOME REASON!!!
"""

def print_clust(clust,labels=None,n=0):
    #indent to make a hierarchy layout
        
    for i in range(n):
        print(' ')
        print(n)
    if clust.id<0:
        #negative id means it is a branch
        print('-')
    else:
        #positive id means it is an endpoint
        if labels==None:
            print(clust.id)
        else:
            print(labels[clust.id])
    
    #now print left and right branches
    if clust.left!=None:
        print_clust(clust.left,labels=labels,n=n+1)
    if clust.right!=None:
        print_clust(clust.right,labels=labels,n=n+1)

In [20]:
print_clust(clust,labels=blog_names)

-
 
1
-
 
2
 
2
-
 
3
 
3
 
3
Wonkette
 
3
 
3
 
3
Wonkette
 
2
 
2
-
 
3
 
3
 
3
Wonkette
 
3
 
3
 
3
Wonkette
 
1
-
 
2
 
2
-
 
3
 
3
 
3
Wonkette
 
3
 
3
 
3
Wonkette
 
2
 
2
-
 
3
 
3
 
3
-
 
4
 
4
 
4
 
4
-
 
5
 
5
 
5
 
5
 
5
Wonkette
 
5
 
5
 
5
 
5
 
5
Wonkette
 
4
 
4
 
4
 
4
-
 
5
 
5
 
5
 
5
 
5
-
 
6
 
6
 
6
 
6
 
6
 
6
Wonkette
 
6
 
6
 
6
 
6
 
6
 
6
Wonkette
 
5
 
5
 
5
 
5
 
5
-
 
6
 
6
 
6
 
6
 
6
 
6
-
 
7
 
7
 
7
 
7
 
7
 
7
 
7
-
 
8
 
8
 
8
 
8
 
8
 
8
 
8
 
8
Wonkette
 
8
 
8
 
8
 
8
 
8
 
8
 
8
 
8
Wonkette
 
7
 
7
 
7
 
7
 
7
 
7
 
7
-
 
8
 
8
 
8
 
8
 
8
 
8
 
8
 
8
-
 
9
 
9
 
9
 
9
 
9
 
9
 
9
 
9
 
9
-
 
10
 
10
 
10
 
10
 
10
 
10
 
10
 
10
 
10
 
10
-
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
Wonkette
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
Wonkette
 
10
 
10
 
10
 
10
 
10
 
10
 
10
 
10
 
10
 
10
-
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
Wonkette
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
 
11
Wonkette
 


17
 
17
 
17
 
17
 
17
 
17
 
17
 
17
 
17
 
17
-
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
-
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
Wonkette
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
Wonkette
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
 
18
-
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
-
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
Wonkette
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
Wonkette
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
 
19
-
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
 
20
-
 
21
 
21
 
21
 
21
 
21
 
21
 
21
 
21
 
21

In [13]:
from PIL import Image,ImageDraw

In [14]:
#first need to get total height of a given cluster
#when determining overall height, need to find total heights
#if cluster is an endpoint, height = 1, otherwise is sum of heights of
#  its branches

def get_height(clust):
    #is this an endpoint? if so the height is 1
    if clust.left == None and clust.right == None:
        return 1
    
    #otherwise height is the sum of the heights of each branch
    return get_height(clust.left) + get_height(clust.right)

In [15]:
#also need to know the total 'error' of the root node
#the length of lines will be scaled to how much error is in each node
#need to generate a scaling factor based on how much total error is there

#error depth of a node is just the maximum possible error from each of
#its branches

def get_depth(clust):
    #the distance of an endpoint is 0.0
    if clust.left == None and clust.right == None:
        return 0
    
    #the distance of a branch is the greater of the two sizes plus
    #its own distance
    return max(get_depth(clust.left),get_depth(clust.right))+clust.distance

In [16]:
#draw_dendrogam function creates a new image allowing 20 pixels in height
#and a fixed width for eaxch final cluster

#the scaling factor is determined by dividing the fixed width by the 
#total depth

#the function creates a 'draw object' for this image and then calls
#draw_node on the root node, tell it that its location should be 
#halfway down the left size of the image

def draw_dendrogram(clust,labels,jpeg='clusters.jpg'):
    #height and width
    h=get_height(clust)*20
    w=1200
    depth=get_depth(clust)
    
    #width is fixed, so scale distances accordingly
    scaling=(w-150)/depth
    
    #create new image with white background
    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)
    
    draw.line((0,h/2,10,h/2),fill=(255,0,0))
    
    #draw the first node
    draw_node(draw,clust,10,(h/2),scaling,labels)
    img.save(jpeg,'JPEG')

In [17]:
#draw_node takes a cluster and location
#it takes the height of the childs and calcs. where they should be
#and draws lines to them, one long vertical and two horizontal

#lengths of the horizontal lines are determined by how much error is 
#in the cluster

#longer lines show that the two clusters that were merged to create the
#cluster weren't all that similar
#shorter lines show they were almost identical

def draw_node(draw,clust,x,y,scaling,labels):
    if clust.id<0:
        h1=get_height(clust.left)*20
        h2=get_height(clust.right)*20
        top=y-((h1+h2)/2)
        bottom=y+((h1+h2)/2)
        #line length
        ll=clust.distance*scaling
        #vertical line from this cluster to children
        draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))
        
        #horizontal line to left item
        draw.line((x,top+h1/2,x+ll,top+h1/2),fill=(255,0,0))
        
        #horizontal line to right item
        draw.line((x,bottom-h2/2,x+ll,bottom-h2/2),fill=(255,0,0))
        
        #call the function to draw the left and right nodes
        draw_node(draw,clust.left,x+ll,top+h1/2,scaling,labels)
        draw_node(draw,clust.right,x+ll,bottom-h2/2,scaling,labels)
        
    else:
        #if this is an endpount, draw the item label
        draw.text((x+5,y-7),labels[clust.id],(0,0,0))

In [18]:
draw_dendrogram(clust,blog_names,jpeg='blog_clust.jpg')